In [1]:
import chess
import chess.engine
import random
import numpy as np

In [2]:
# this function will create our x (board)
def random_board_generator(max_depth=200):
  board = chess.Board()
  depth = random.randrange(0, max_depth)

  for _ in range(depth):
    all_moves = list(board.legal_moves)
    random_move = random.choice(all_moves)
    board.push(random_move)
    if board.is_game_over():
      break

  return board

In [3]:
# this function will create our f(x) (score)
def stockfish(board, engine, depth):
#  engine = chess.engine.SimpleEngine.popen_uci("/home/dsc/Data_Science_Projects/Chess_Engine_TFM/engine/stockfish_15.1_linux_x64_bmi2/stockfish_15.1_x64_bmi2")
#  engine = chess.engine.SimpleEngine.popen_uci(r"C:\Users\Usuario\Desktop\Data Science Projects\Chess_Engine_TFM\engine\stockfish15\stockfish-windows-2022-x86-64-avx2.exe")
  result = engine.analyse(board, chess.engine.Limit(depth=depth))
  score = result['score'].white().score()

  return score

In [4]:
# example: h3 -> 17
def square_to_index(square):
  squares_index = {'a': 0,
                   'b': 1,
                   'c': 2,
                   'd': 3,
                   'e': 4,
                   'f': 5,
                   'g': 6,
                   'h': 7}
                   
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = np.zeros((14, 8, 8), dtype=np.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = np.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = np.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

In [9]:
def dataset_generator(n_games, save_path, engine_path, stockfish_depth=10):
    board_matrix_list =[]
    eval_list = []
    fen_list = []

    engine = chess.engine.SimpleEngine.popen_uci(engine_path)

    for i in range(0, n_games):
        board = random_board_generator()
        matrix = split_dims(board)
        eval = stockfish(board, engine=engine, depth=stockfish_depth)
        fen = board.fen()
        board_matrix_list.append(matrix)
        eval_list.append(eval)
        fen_list.append(fen)
        
        if int(i) % 1000 == 0: print(i)

    np.savez(save_path, board_matrix=board_matrix_list, eval=eval_list, board_fen=fen_list)
    engine.quit()


In [11]:
outfile = r'C:\Users\Usuario\Desktop\Data Science Projects\Chess_Engine_TFM\engine\random_boards_d3.npz'
stockfish_path = (r"C:\Users\Usuario\Desktop\Data Science Projects\Chess_Engine_TFM\engine\stockfish15\stockfish-windows-2022-x86-64-avx2.exe")
dataset_generator(1_500_000, save_path=outfile, engine_path=stockfish_path, stockfish_depth=3)

0
1000
2000
3000
4000


In [ ]:
outfile = r'C:\Users\Usuario\Desktop\Data Science Projects\Chess_Engine_TFM\engine\random_boards_d10.npz'
stockfish_path = (r"C:\Users\Usuario\Desktop\Data Science Projects\Chess_Engine_TFM\engine\stockfish15\stockfish-windows-2022-x86-64-avx2.exe")
dataset_generator(1_500_000, save_path=outfile, engine_path=stockfish_path, stockfish_depth=10)

0
1000
2000
3000
4000


In [ ]:
outfile = r'C:\Users\Usuario\Desktop\Data Science Projects\Chess_Engine_TFM\engine\random_boards_d6.npz'
stockfish_path = (r"C:\Users\Usuario\Desktop\Data Science Projects\Chess_Engine_TFM\engine\stockfish15\stockfish-windows-2022-x86-64-avx2.exe")
dataset_generator(1_500_000, save_path=outfile, engine_path=stockfish_path, stockfish_depth=6)

0
1000
2000
3000
4000
